<h3>Глобальные переменные</h3>

In [1]:
import torch
#путь до языковой модели, которая будет учиться в составе сетки. Полный список в https://huggingface.co/models
transformer_model_path = 'DeepPavlov/rubert-base-cased'
#Использовать ли видеокарты. torch может автоматически определить эту переменную
USE_CUDA = torch.cuda.is_available()

<h3>Векторизуем словарь Meddra</h3>

In [2]:
from vectorization import ConceptVectorizer

In [3]:
#сюда прописывается путь до модели и путь до файла словаря расширения .asc, который необходимо векторизовать
#для того, чтобы использовать векторизатор без векторизационной модели, 
#например, когда готовые векторы концептов уже есть, и модель использовать не нужно, ставьте use_model=False
CV = ConceptVectorizer(transformer_model_path, '../../Data/External/pt_rus.asc', \
                             use_concept_less=False, use_model=True, use_cuda=USE_CUDA)

Расчет вложений словаря

In [5]:
#используется только для векторизации словаря
#когда вектора готовы, проще их загрузить ячейкой с загрузкой
CV.fit_transform(mode='mean_pooling')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


getting concept embeddings in mean_pooling mode...
Compute embeddings...
in cpu mode there is no progress bar
Embedding aggregation...
Concept embeddings have computed in 251.15103459358215 seconds


Сохранение рассчитанных векторов концептов

In [5]:
torch.save(CV.thesaurus_embeddings, './Model_weights/rubert_thesaurus_embeddings_meddra_origin.pt')

Загрузка готовых вложений (если они есть). Загружать вложения после fit_transform() не надо

In [4]:
CV.thesaurus_embeddings = torch.load('./Model_weights/rubert_thesaurus_embeddings_meddra_origin.pt') 
CV.normalization_mode = 'mean_pooling'

<h3>Обработка датасета, пример с RDRS</h3>

In [5]:
import jsonlines
import numpy as np
from sklearn.model_selection import train_test_split

In [6]:
ds = []
with jsonlines.open('../../Data/Raw/medNorm_16022022.jsonlines') as reader:
    for obj in reader:
        ds.append(obj)

In [7]:
X_train, X_test = train_test_split(ds, test_size=0.33, random_state=42)

In [8]:
#выцепим фразы с нормализацией по Meddra без их контекста
train_phrases = []
train_concepts = []

test_phrases = []
test_concepts = []

train_ids = []
test_ids = []

USE_CONCEPT_LESS=False

for review in X_train:
    for ent in review['objects']['MedEntity']:
        if 'MedDRA' in ent.keys():
            if ent['MedDRA']=='':
                if USE_CONCEPT_LESS:
                    ent['MedDRA'] = 'CONCEPT_LESS'
                else:
                    continue
            train_concepts.append(CV.meddra_term_to_meddra_code[ent['MedDRA'].split('|')[0]])
            train_phrases.append(ent['text'])
            train_ids.append(review['meta']['fileName'])
            
for review in X_test:
    for ent in review['objects']['MedEntity']:
        if 'MedDRA' in ent.keys():
            if ent['MedDRA']=='':
                if USE_CONCEPT_LESS:
                    ent['MedDRA'] = 'CONCEPT_LESS'
                else:
                    continue
            test_concepts.append(CV.meddra_term_to_meddra_code[ent['MedDRA'].split('|')[0]])
            test_phrases.append(ent['text'])

In [9]:
from dataset import MedNormDataset

ds_train = MedNormDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
ds_test = MedNormDataset(test_phrases, test_concepts, CV, use_cuda=USE_CUDA)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


<h2>Обучение</h2>

<h4>Настройка глобальных переменных для большей детерменированности</h4>

In [14]:
import os 

torch.manual_seed(0)
np.random.seed(0)

<h4>Импорт модели</h4>

In [11]:
from models import CADEC_SoTa


net = CADEC_SoTa(transformer_model_path, CV.thesaurus_embeddings)
#перемещение модели на вычислительные мощности, либо cpu, либо cuda
device = 'cuda' if USE_CUDA else 'cpu'
net.to(device)
print('Net loaded')

Some weights of the model checkpoint at ./bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Net loaded


<h4>Инициализация доп. объектов для обучения</h4>

In [12]:
import torch.optim as optim
import torch.nn as nn
import torch
import numpy as np
from evaluator import Evaluator
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.0001)
#scheduler = CyclicLR(optimizer, base_lr=4.7e-05, max_lr=0.0002, cycle_momentum=False) если с циклическим lr
scaler = torch.cuda.amp.GradScaler()
evaluator = Evaluator(train_concepts, test_concepts)

<h3>Процесс обучения и рассчета точностей</h3>

In [ ]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score

#Гиперпараметры
batch_size=16
epochs = 1
#номер эксперимента
exp_num = 11
    
trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)

net.train()
initial_loss = None
for epoch in range(1, epochs+1):
    #обучение модели в эпохе
    net.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:

            tepoch.set_description(f"Epoch {epoch}")

            inputs = data['tokenized_phrases']
            labels = data['one_hot_labels']

            optimizer.zero_grad()
            if USE_CUDA:
                with torch.cuda.amp.autocast():
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = net(inputs)['output']
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            if initial_loss is None:
                initial_loss = loss.item()
            tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #расчет точностей модели после эпохи
    net.eval()
    model_answers=[]
    real_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]


            model_answers.append(pred_meddra_code)
            real_answers.append(data['label_codes'])
    
    evaluator.get_all_f1_scores(real_answers, model_answers)
            
print('Finished Training')
torch.save(net,'./Model_weights/rubert_exp_%s.pt'%exp_num)

Инференс (подать фразу в сетку и посмотреть, какой концепт она выдаст)

Чтобы сетка выдавала отсутствие концепта на несвязанные с медрой фразы, раскомментируйте строчку с label_concepless_tensors

In [15]:
from transformers import AutoTokenizer

phrase = 'Болезнь'
tokenizer = AutoTokenizer.from_pretrained(transformer_model_path)
encoded_input = tokenizer([phrase], padding=True, truncation=True, return_tensors='pt')

with torch.no_grad():
    outputs_dict = net(inputs)
    #outputs_dict.label_concepless_tensors(score_treshold = 6.1977e-05)
    pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]
    
print('phrase: %s'%phrase)
print('model output: %s'%CV.meddra_code_to_meddra_term[pred_meddra_code])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


phrase: Болезнь
model output: Повышенная температура тела


<h2>Сохранение и загрузка модели</h2>

сохранение

In [63]:
torch.save(net, './trained_model_weights.pt')
torch.save(optimizer.state_dict(), './trained_model_opt.pt')

загрузка

In [64]:
the_model = torch.load('./trained_model_weights.pt')